In [3]:
# ============================================================
# Robust merge for VAR / Spillover datasets (handles numeric-as-text)
# Output: ./merged_spillover_data.csv
# ============================================================

import re
import pandas as pd

FILES = {
    "SOLVPN": "./SOLVPN_index.csv",
    "COPPER": "./copper_futures.csv",
    "DXY": "./dollar_index.csv",
    "UST10Y": "./us_10y_bond_yield.csv",
    "VIX": "./cboe_vix_index.csv"
}

PREFERRED_VALUE_COLS = [
    "adj close", "adjusted close", "close", "price", "last", "value", "index", "settle"
]

def _find_date_col(cols):
    # 흔한 날짜 컬럼 패턴 우선 탐색
    candidates = [c for c in cols if re.search(r"(date|time|timestamp)", c, re.IGNORECASE)]
    if candidates:
        return candidates[0]
    # 그래도 없으면 첫 컬럼을 날짜로 가정(최후 수단)
    return cols[0]

def _to_numeric_series(s: pd.Series) -> pd.Series:
    # 문자열 숫자(콤마, %, 공백 등) 정리 후 numeric 변환
    if s.dtype == "O":
        s = (
            s.astype(str)
             .str.strip()
             .str.replace(",", "", regex=False)
             .str.replace("%", "", regex=False)
        )
    return pd.to_numeric(s, errors="coerce")

def load_and_clean(path, series_name):
    df = pd.read_csv(path)

    # 1) Date 컬럼 찾기
    date_col = _find_date_col(df.columns.tolist())
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col]).sort_values(date_col)

    # 2) 값 컬럼 후보 찾기: 선호 컬럼명 우선
    lower_map = {c.lower(): c for c in df.columns}
    chosen_value_col = None

    for key in PREFERRED_VALUE_COLS:
        # 정확 일치
        if key in lower_map:
            chosen_value_col = lower_map[key]
            break
        # 부분 일치
        for lc, orig in lower_map.items():
            if key in lc and orig != date_col:
                chosen_value_col = orig
                break
        if chosen_value_col is not None:
            break

    # 3) 선호 컬럼 못 찾으면: Date 제외 모든 컬럼에 대해 numeric 변환 성공률로 고르기
    if chosen_value_col is None:
        candidates = [c for c in df.columns if c != date_col]
        best_col, best_ratio = None, -1.0

        for c in candidates:
            numeric_s = _to_numeric_series(df[c])
            ratio = numeric_s.notna().mean()  # 숫자 변환 성공 비율
            if ratio > best_ratio:
                best_ratio = ratio
                best_col = c

        if best_col is None or best_ratio <= 0.05:
            raise ValueError(
                f"[{series_name}] numeric-like column not found. "
                f"Columns={df.columns.tolist()}"
            )

        chosen_value_col = best_col

    # 4) 최종 값 컬럼 numeric 변환
    val = _to_numeric_series(df[chosen_value_col])
    out = pd.DataFrame({"Date": df[date_col], series_name: val}).dropna(subset=[series_name])

    # 5) 중복 날짜 처리(있으면 마지막 값 사용)
    out = out.sort_values("Date").drop_duplicates("Date", keep="last").reset_index(drop=True)

    print(f"[{series_name}] date_col='{date_col}', value_col='{chosen_value_col}', rows={len(out)}")
    return out

# ----------------------------
# Load all series
# ----------------------------
dfs = [load_and_clean(path, name) for name, path in FILES.items()]

# ----------------------------
# Merge (inner join: 공통 날짜만)
# ----------------------------
merged = dfs[0]
for df in dfs[1:]:
    merged = merged.merge(df, on="Date", how="inner")

merged = merged.sort_values("Date").reset_index(drop=True)

print("\n[Merged]")
print("shape:", merged.shape)
print("date range:", merged["Date"].min(), "~", merged["Date"].max())
print("columns:", merged.columns.tolist())

OUT_PATH = "./merged_spillover_data.csv"
merged.to_csv(OUT_PATH, index=False)
print("Saved to:", OUT_PATH)


[SOLVPN] date_col='Date', value_col='Close', rows=1344
[COPPER] date_col='Date', value_col='Close', rows=2618
[DXY] date_col='Date', value_col='Close', rows=2595
[UST10Y] date_col='Date', value_col='Close', rows=2619
[VIX] date_col='Date', value_col='Close', rows=1558

[Merged]
shape: (1326, 6)
date range: 2020-10-12 00:00:00 ~ 2026-01-12 00:00:00
columns: ['Date', 'SOLVPN', 'COPPER', 'DXY', 'UST10Y', 'VIX']
Saved to: ./merged_spillover_data.csv
